In [1]:
import pandas as pd
import numpy as np
import tensorflow-cpu as tf

In [2]:
data = pd.read_csv('stock_codes.csv')
stockcode_description = pd.read_csv('stock_code_to_desc.csv')

Y = data['StockCode']

unique_stockcodes = np.unique(Y)
Y[0] == unique_stockcodes
boolean_stockcodes = [StockCode == unique_stockcodes for StockCode in Y]

y = boolean_stockcodes

In [3]:
def create_batches(X, y=None, batch_size=32, valid_data=False, test_data=False):
    if test_data:
#         print("Creating test batches...")
        data = tf.data.Dataset.from_tensor_slices(tf.constant(X))
        data_batch = data.batch(batch_size)
        return data_batch
    elif valid_data:
#         print("Creating validation batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                                  tf.constant(y)))
        data_batch = data.batch(batch_size)
        return data_batch
    else:
#         print("Creating training batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                                 tf.constant(y)))
        data = data.shuffle(buffer_size=len(X))
        data_batch = data.batch(batch_size)
        return data_batch

In [4]:
model = tf.keras.models.load_model('best_model.h5')

In [5]:
def get_pred_stockcode(prediction_matrix):
    return unique_stockcodes[np.argmax(prediction_matrix)]

In [6]:
def getKey(item):
    return item[0]

In [72]:
def prediction_dataframe(prob_matrix, top_five=True):
    prob_matrix = prob_matrix[0]
    matrix = np.array((prob_matrix, unique_stockcodes))
    matrix = matrix.T
    testing = tuple(map(tuple, matrix))
    if top_five:
        testing_2 = sorted(testing, key=getKey, reverse = True)
    else:
        testing_2 = sorted(testing, key=getKey, reverse = False)
    testing_3 = testing_2[:5]
    data = pd.DataFrame(testing_3, columns=['Prediction','Item Code'])
    data2 = data['Item Code']
    nparray = [[0,0]]
    for x in range(5):
        guess_frame = stockcode_description[stockcode_description['StockCode'].str.contains(str(data2[x]))].values
        nparray = np.append(nparray,(guess_frame[0][0],guess_frame[0][1]))
    testing_4 = nparray[2:]
    data2 = np.reshape(testing_4,(5,2))
    data2 = pd.DataFrame(data2, columns=['Item Code', 'Description'])
    newest = pd.concat([data,data2[['Description']]], axis=1, join="inner")
    return newest

In [73]:
def output_guess(rain,mean_temp,min_temp,max_temp):
    test_me_data = [[rain,mean_temp,min_temp,max_temp]]
    test_me_data = np.c_[test_me_data[:]]
    
    test_me_data = create_batches(test_me_data, test_data=True)
    attempt = model.predict(test_me_data, verbose=0)
    guess = get_pred_stockcode(attempt[0])
    guess = str(guess)
    
    stockcode_description = pd.read_csv('stock_code_to_desc.csv')
    
    guess_frame = stockcode_description[stockcode_description['StockCode'].str.contains(str(guess))]
    
    guess_numpy = guess_frame.to_numpy()
    stock, desc = guess_numpy[0][0],guess_numpy[0][1]
    print("Top Five Items")
    print(prediction_dataframe(attempt, top_five = True))
    print("Bottom Five Items")
    print(prediction_dataframe(attempt, top_five = False))
    return stock, desc, attempt

In [74]:
stock, desc, attempt = output_guess(11,70,20,120)
print("The model predicts the sales of Stock Code: " + stock + " which is product: " + desc)

Top Five Items
   Prediction Item Code                         Description
0    0.003156     22827  RUSTIC  SEVENTEEN DRAWER SIDEBOARD
1    0.001808     85161              ACRYLIC GEOMETRIC LAMP
2    0.001740     22833         HALL CABINET WITH 3 DRAWERS
3    0.001603     21316      SMALL CHUNKY GLASS ROMAN  BOWL
4    0.001589    90214K           LETTER "K" BLING KEY RING
Bottom Five Items
   Prediction Item Code                        Description
0    0.000055     21785                       RAIN PONCHO 
1    0.000104     22632          HAND WARMER RED POLKA DOT
2    0.000113    90178A   AMBER CHUNKY GLASS+BEAD NECKLACE
3    0.000117     20878  SET/9 CHRISTMAS T-LIGHTS SCENTED 
4    0.000130     23191   BUNDLE OF 3 RETRO EXERCISE BOOKS
The model predicts the sales of Stock Code: 22827 which is product: RUSTIC  SEVENTEEN DRAWER SIDEBOARD


In [75]:
prediction_dataframe(attempt, top_five = True)

,Prediction,Item Code,Description
0,0.003156,22827,RUSTIC SEVENTEEN DRAWER SIDEBOARD
1,0.001808,85161,ACRYLIC GEOMETRIC LAMP
2,0.001740,22833,HALL CABINET WITH 3 DRAWERS
3,0.001603,21316,SMALL CHUNKY GLASS ROMAN BOWL
4,0.001589,90214K,"LETTER ""K"" BLING KEY RING"


In [76]:
from ipywidgets import interactive
interactive_plot = interactive(output_guess, rain=(0,15.2,0.1), mean_temp=(-70,220,10.0),
                              min_temp=(-120,160,10.0), max_temp=(-30,280,10.0))

#### Our interactive model

Finally, we have an output model that has an interactive slider system to take weather data as input and output predicted stock codes and respective description

In [77]:
print()
interactive_plot

interactive(children=(FloatSlider(value=7.5, description='rain', max=15.2), FloatSlider(value=70.0, descriptio…

*footnote: All temperatures are in 1/10th degrees Celcius (ie. 100 mean_temp is 10.0 degrees Celcius)